In [1]:
import os, sys, pathlib

# ← change this if you unzipped somewhere else
project_dir = pathlib.Path.home() / "Downloads" / "10k-rag"
os.chdir(project_dir)
print("CWD:", os.getcwd())

if str(project_dir) not in sys.path:
    sys.path.insert(0, str(project_dir))

# ---- install only what you need ----
!pip install -U pip
!pip install torch torchvision torchaudio
!pip install sentence-transformers beautifulsoup4 faiss-cpu
!pip install langchain==0.3.* langchain-community==0.3.*
!pip install pypdf tiktoken fastapi uvicorn datasets evaluate ragas python-dotenv

# optional check for Apple Metal acceleration
import torch
print("MPS available:", torch.backends.mps.is_available())

CWD: /Users/julianp/Downloads/10k-rag
zsh:1: no matches found: langchain==0.3.*
MPS available: True


In [2]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3.1:8b", temperature=0)

/Users/julianp/Downloads/10k-rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/g2/yw_64wy94kv8y7x___dkn5hm0000gn/T/ipykernel_61436/873696594.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1:8b", temperature=0)


In [3]:
import glob, os
files = glob.glob("data/raw/*.pdf") + glob.glob("data/raw/*.htm") + glob.glob("data/raw/*.html") + glob.glob("data/raw/*.xml")
print("Found:", len(files))
for p in files:
    print(" -", os.path.basename(p))

Found: 3
 - apple_10k_2025.html
 - tesla_10k_2025.html
 - jpm_10k_2025.html


In [12]:
import os
!python -m app.ingest

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
Indexed 2779 chunks -> data/processed/faiss


In [13]:
import os
print(os.listdir("data/processed/faiss"))

['index.faiss', 'index.pkl']


In [14]:
import glob
print(glob.glob("data/raw/*"))

['data/raw/apple_10k_2025.html', 'data/raw/tesla_10k_2025.html', 'data/raw/jpm_10k_2025.html']


In [15]:
from app.rag import get_retriever
retriever = get_retriever()
docs = retriever.invoke("supply chain")
print(len(docs))
print(docs[0].page_content[:500])

5
supply chain and sales and distribution channels, resulting in interruptions to the supply of current products and offering of existing services, and delays in production ramps of new products and development of new services.


In [16]:
DATA_RAW = "data/raw"
DB_DIR = "data/processed/faiss"

In [17]:
from app.rag import build_chain
chain = build_chain()

q = "What supply chain risks does Tesla mention?"
resp = chain.invoke(q)
print(getattr(resp, "content", resp))

Based on the provided context, Tesla mentions the following supply chain risks:

1. Disruptions caused by natural disasters such as earthquakes [apple_10k_2025.html#pX] (Note: The exact page number is not specified in the apple_10k_2025.html file, but it's mentioned in the context).
2. Interruptions by fire, power shortages, nuclear power plant accidents, and other industrial accidents [apple_10k_2025.html].
3. Terrorist attacks and other hostile acts [apple_10k_2025.html].
4. Ransomware and other cybersecurity attacks [apple_10k_2025.html].
5. Labor disputes [tesla_10k_2025.html#pX] (Note: The exact page number is not specified in the tesla_10k_2025.html file, but it's mentioned in the context).
6. Public health issues [apple_10k_2025.html].
7. Geopolitical tensions or conflict resulting in significant disruptions to global supply chains [apple_10k_2025.html].
8. Strikes organized by unions among suppliers and trucking and freight companies [tesla_10k_2025.html#pX] (Note: The exact pa

In [18]:
q = "According to Tesla's 10-K, what are the supply chain risks mentioned? Only use data from tesla_10k_2025.html."
resp = chain.invoke(q)
print(getattr(resp, "content", resp))

According to Tesla's 10-K, the supply chain risks mentioned are:

* Industry-wide shortages and significant commodity pricing fluctuations that can materially adversely affect the Company's business, results of operations, and financial condition [apple_10k_2025.html#pX]
* No assurance that Tesla will be able to extend or renew agreements with component suppliers on similar terms, or at all [apple_10k_2025.html#pX]
* Component suppliers may suffer from poor financial health, which could impact their ability to supply components to Tesla [apple_10k_2025.html#pX]

Note that the provided context does not mention specific supply chain risks for Tesla. The text mentions general risks and challenges faced by the company, but it does not specifically address supply chain risks.

However, based on the information in the 10-K, we can infer some potential supply chain risks for Tesla:

* Dependence on companies with unionized workforces, such as suppliers and trucking and freight companies, whic

In [19]:
# --- Ask company-specific questions about each 10-K ---

from app.rag import build_chain
chain = build_chain()

# Define company-specific queries
company_queries = {
    "apple_10k_2025.html": [
        "What supply chain risks does Apple mention?",
        "How does Apple describe its dependence on key suppliers or manufacturing partners?",
        "What are Apple’s main sources of revenue?",
        "What legal or regulatory risks does Apple discuss?",
        "What sustainability or environmental initiatives are mentioned?"
    ],
    "jpm_10k_2025.html": [
        "What market risks does JPMorgan identify?",
        "How does JPMorgan describe its risk management framework?",
        "What does JPMorgan report about its total assets and liabilities?",
        "What operational risks such as cybersecurity or fraud are mentioned?",
        "What growth opportunities or challenges are discussed in investment banking?"
    ]
}

# Run each query per company
for company_file, questions in company_queries.items():
    print("="*100)
    print(f"🔍 {company_file.replace('_10k_2025.html', '').upper()} — Questions and Answers")
    print("="*100)

    for q in questions:
        # Add file filter hint to help retrieval focus on that document
        query = f"Answer based only on {company_file}. {q}"
        print(f"\n❓ {q}\n")
        resp = chain.invoke(query)
        print(f"🧾 {getattr(resp, 'content', resp)}")
        print("-"*100)

🔍 APPLE — Questions and Answers

❓ What supply chain risks does Apple mention?

🧾 Based on the provided context from apple_10k_2025.html, Apple mentions the following supply chain risks:

1. Design and manufacturing defects that could materially adversely affect their business and reputation [apple_10k_2025.html#8].
2. Supply chain disruptions due to reliance on single or limited sources for critical components, which could exacerbate negative consequences [apple_10k_2025.html#6].
3. Unfavorable results of legal proceedings or government investigations that could impact their business and financial condition [apple_10k_2025.html#12].

Additionally, Apple mentions the following risks related to their supply chain:

1. Higher net sales in the first quarter due to seasonal holiday demand, which can create uncertainty in other quarters [apple_10k_2025.html#45].
2. Dependence on a single product for a significant portion of net sales, which could be impacted by changes in demand [apple_10k_